# Riverkeeper Data: Evaluating Enterococcus in NY Rivers

For this homework we decided to utilize both Pandas and Bokeh to analyse the data. In brief, the data itself was relatively clean, however, it was necessary to convert some non-numeric data. In 4 instances, the data had either a > or < point. These were: [">2420", ">24196","<1","<10"]. Out of necessity these points were converted to a "rounded" number, arbitrarly set as:[2500,25000,0,5] respectively. This was done to make it easier to manipulate the data.

## Question 1 

In [162]:
import pandas as pd
import numpy as np 

#Data Upload
url = "https://raw.githubusercontent.com/jlaurito/CUNY_IS608/master/lecture4/data/riverkeeper_data_2013.csv"
dat_raw = pd.read_csv(url, index_col = 0)

# Data Cleaning (removine Greater than and Less then values)
rep_val = [">2420", ">24196","<1","<10"]
new_val = [2500,25000,0,5]
dat_raw["EnteroCount"] = dat_raw["EnteroCount"].replace(rep_val,new_val)
dat_raw["EnteroCount"] = dat_raw["EnteroCount"].astype(int)
dat_raw['Date'] = pd.to_datetime(dat_raw['Date'])

#Create some datasets for displaying values
dat_ent = dat_raw.drop(['FourDayRainTotal','SampleCount'], 1)
dat_avg = dat_raw.groupby(dat_ent.index).mean()


For Question 1, it was decided that finding the Top and Bottom 10 places representing the best and worse places to swim by taking averages over all the samples, as there were many data points per location. This was used as a starting point for our analysis. Below is the displayed results: 

In [163]:
bot_10 = dat_avg.sort_values('EnteroCount', ascending=False).head(10)
bot_10

,EnteroCount,FourDayRainTotal,SampleCount
Site,,,
Gowanus Canal,4314.540541,0.816216,37
Newtown Creek- Metropolitan Ave. Bridge,3037.614035,0.852632,57
Tarrytown Marina,2264.481481,0.814815,27
Saw Mill River,1471.540000,0.742000,50
Upper Sparkill Creek,1315.515152,0.526061,165
Newtown Creek- Dutch Kills,1231.807018,0.852632,57
Kingsland Pt. Park- Pocantico River,924.976190,0.602381,42
Orangetown STP Outfall,867.596491,0.615789,57
Mohawk River at Waterford,625.628571,0.631429,35


In [164]:
top_10 = dat_avg.sort_values('EnteroCount', ascending=True).head(10)
top_10

,EnteroCount,FourDayRainTotal,SampleCount
Site,,,
Poughkeepsie Drinking Water Intake,8.210526,0.350000,38
Croton Point Beach,13.062500,0.464583,48
Stony Point mid-channel,15.068182,0.461364,44
Haverstraw Bay mid-channel,16.104167,0.466667,48
Little Stony Point,17.394737,0.368421,38
Poughkeepsie Launch Ramp,17.675676,0.318919,37
TZ Bridge mid-channel,18.456140,0.701754,57
Yonkers mid-channel,22.326923,0.736538,52
Cold Spring Harbor,22.514286,0.305714,35


In [165]:
# Taking the Index for the top and bottom 10 Rivers
top_10_ind = top_10.index.tolist()
bot_10_ind = bot_10.index.tolist()

# Creating a Year Column then Average over that year, and Pivotting the data
dat_ent['Year'] = dat_ent['Date'].dt.year
dat_yearly_bot = dat_ent.loc[bot_10_ind,:]
dat_yearly_bot.reset_index(level=0, inplace=True)

dat_yearly_top = dat_ent.loc[top_10_ind,:]
dat_yearly_top.reset_index(level=0, inplace=True)


In this first graph, we wanted to show the outliers, as an explanation needs to be done regarding the areas with an Entero Count greater than 24196. Because we set these values equal to 25000 in the beginning, we can see how these outliers are significant. First, it greatly increased the average of these specific site, and as you can tell almost all of the bottom 10 had at one point this astronomically high reading. Arguably, any place that has such a high reading should AUTOMATICALLY be a no-swim zone, so it would be accurate to say that these graphs  

In [166]:
from bokeh.charts import *

p = BoxPlot(dat_yearly_bot, 
            values='EnteroCount', 
            label="Site",  
            color = "Site",
            title="Worse Places to Swim Based on Entero Count ")

output_notebook()

show(p)

Loading BokehJS ...

After reviewing the outlier data, we decided to remove it to compare our top 10 to our bottom ten, which we can see below: 

In [167]:
p = BoxPlot(dat_yearly_bot, 
            values='EnteroCount', 
            label="Site",  
            color = "Site",
            outliers= False, 
            title="Worse Places to Swim Based on Entero Count ")

output_notebook()

show(p)

Loading BokehJS ...

In [168]:
p = BoxPlot(dat_yearly_top,
            values='EnteroCount', 
            label="Site",
            color = "Site",
            outliers= False, 
            title="Best Places to Swim Based on Entero Count ")

output_notebook()

show(p)

Loading BokehJS ...

## Question 2 

The next area we wish to test is the frequency between sampling. First, we want to get an understanding which areas were tested most, so using the sample count, we compared the highest and lowest sample counts:  

In [169]:
#Technically.... This took the Mean of the sample count... But that works too...
dat_raw.groupby(dat_raw.index).mean().sort_values('SampleCount', ascending=False).head(5)

,EnteroCount,FourDayRainTotal,SampleCount
Site,,,
Piermont Pier,481.577540,0.533690,187
Upper Sparkill Creek,1315.515152,0.526061,165
125th St. Pier,178.787879,0.771212,66
Nyack Launch Ramp,103.803279,0.506557,61
Newtown Creek- Dutch Kills,1231.807018,0.852632,57


In [170]:
dat_raw.groupby(dat_raw.index).mean().sort_values('SampleCount', ascending=True).head(5)

,EnteroCount,FourDayRainTotal,SampleCount
Site,,,
Tarrytown Marina,2264.481481,0.814815,27
Cold Spring Harbor,22.514286,0.305714,35
Hudson above Mohawk River,228.200000,0.631429,35
Island Creek/Normans Kill,438.685714,0.642857,35
Marlboro Landing,87.428571,0.291429,35


As you can see, most areas were sampled at least 35 times (there is only one 27 sample in the group) Piermont Pier and Upper Sparkill Creek, were definitely sampled more frequently than any other, by a large margin. It is also interesting to note that Upper Sparkill was listed as one of the bottom places to swim in the previous section.

Next, using pandas, we found the average difference between each sample. This was accomplished by finding the max and minimum date in each area, and then dividing that by the sample number: 

In [171]:
dat_freq = dat_raw.groupby(dat_raw.index).agg({ "SampleCount" : np.mean, "Date" : [np.max,np.min]})
dat_freq["Days"] = dat_freq["Date"]["amax"] -dat_freq["Date"]["amin"]
dat_freq["AvgDiff"] = dat_freq["Days"]/dat_freq["SampleCount"]["mean"]


In [172]:
dat_freq_bot= dat_freq.sort_values('AvgDiff', ascending=False).head(5)
dat_freq_bot

SampleCount       Date                 Days  \
                                 mean       amax       amin             
Site                                                                    
Tarrytown Marina                   27 2013-10-17 2008-08-06 1898 days   
Gowanus Canal                      37 2013-10-16 2007-04-18 2373 days   
Stony Point mid-channel            44 2013-10-17 2006-09-19 2585 days   
Hudson above Mohawk River          35 2013-10-21 2008-05-22 1978 days   
Mohawk River at Waterford          35 2013-10-21 2008-05-22 1978 days   

                                          AvgDiff  
                                                   
Site                                               
Tarrytown Marina                 70 days 07:06:40  
Gowanus Canal             64 days 03:14:35.675675  
Stony Point mid-channel          58 days 18:00:00  
Hudson above Mohawk River 56 days 12:20:34.285714  
Mohawk River at Waterford 56 days 12:20:34.285714

In [173]:
dat_freq_top = dat_freq.sort_values('AvgDiff', ascending=True).head(5)
dat_freq_top

SampleCount       Date                 Days  \
                                    mean       amax       amin             
Site                                                                       
Upper Sparkill Creek                 165 2012-12-10 2006-10-13 2250 days   
Piermont Pier                        187 2013-10-17 2006-10-17 2557 days   
Nyack Launch Ramp                     61 2013-10-17 2008-06-16 1949 days   
125th St. Pier                        66 2013-10-16 2006-10-02 2571 days   
Rondout- Eddyville Anchorage          45 2013-10-19 2008-05-21 1977 days   

                                             AvgDiff  
                                                      
Site                                                  
Upper Sparkill Creek         13 days 15:16:21.818181  
Piermont Pier                13 days 16:10:16.042780  
Nyack Launch Ramp            31 days 22:49:10.819672  
125th St. Pier               38 days 22:54:32.727272  
Rondout- Eddyville Anchorage        43 days 22:24:00

As you can see, Tarrytown Marina was sampled least often, while Upper Sparkil was sampled most often, average 13 days per sample. It is noteworthy, that not all of the lowest sampled areas are reflected in the bottom five. This is likely due to the fact that there was a larger time-frame for which they were sampled. 

Next a couple of graphs were created to show at what time each sample was collected. We can see a very common pattern, especially among the bottom 5:

In [174]:
freq_top_ind = dat_freq_top.index.tolist()
freq_bot_ind = dat_freq_bot.index.tolist()

freq_top = dat_raw.loc[freq_top_ind]
freq_top.reset_index(level=0, inplace=True)
freq_bot = dat_raw.loc[freq_bot_ind]
freq_bot.reset_index(level=0, inplace=True)

In [175]:
p = Scatter(freq_top,
            x='Date',
            y='Site',
            color="Site",
            title="Top 5 Sites Sampled during the Year", 
            xlabel="Year",
            ylabel="Site")
output_notebook()
show(p)

Loading BokehJS ...

In [176]:
p = Scatter(freq_bot,
            x='Date',
            y='Site',
            color="Site",
            title="Bottom 5 Sites Sampled during the Year", 
            xlabel="Year",
            ylabel="Site")
output_notebook()
show(p)

Loading BokehJS ...

As you can see, the largest gaps, even in the areas sampled the most, were over the winter months(the line denotes January). Again, Tarrytown was sampled least, and also has the largest gaps between sampling, some almost a year in duration between 2009 and 2011.

## Question 3

For Question 3, we started our analysis by taking our total raw data, and plotting Rain Totals vs Enterocount to see if we have any apparent relationships that we can focus on: 

In [177]:
p = Scatter(dat_raw, 
            x='EnteroCount', 
            y='FourDayRainTotal', 
            title="Rain Totals vs. Entero Count", 
            color="navy",
            xlabel="Entero Count", 
            ylabel="4 Day Rain Total")
output_notebook()
show(p)

Loading BokehJS ...

As we can see from the above plot most 

Next we compared Average Rain per location: 

In [178]:
dat_rain = dat_raw.groupby(dat_ent.index).mean()
dat_rain.reset_index(level=0, inplace=True)

In [179]:
p = Scatter(dat_rain, 
            x='EnteroCount', 
            y='FourDayRainTotal', 
            title="Average Rain Totals vs. Average Entero Count", 
            color="Site",
            legend = False,
            xlabel="Entero Count", 
            ylabel="4 Day Rain Total")
output_notebook()
show(p)

Loading BokehJS ...

Here we can see it might be more likely that rain (most likely from run-off) may well result in an increase in the entero count. It is likely that there is some relationship between rain and enterocount. We decided to take the top 15 sites with the highest average rain totals, and compare all the datapoints (as we did above). 